## DashBoard TFM

Notebook donde se diseña una app de Streamlit para predecir la cantidad de llamdas a recibir en base a una serie de info

In [1]:
import pandas as pd
import numpy as np
import streamlit as st
import pyodbc
import sys

from sklearn.preprocessing import OneHotEncoder
import datetime
import calendar

import pickle
from sklearn.preprocessing import StandardScaler

# Datos Llamadas

In [2]:
def getCallData(password):
    server = 'khcuy9chlt.database.windows.net' 
    bdd = 'ASERTEC' 
    user = 'tecnico@khcuy9chlt' 
    #password = 'xxxxxxxxxxxx'

    query = 'SELECT [IDCAMPANYA] \
          ,[IDSUJETO] \
          ,[VALOR] \
          ,[DATACREACIO] \
        FROM [dbo].[T_CCT_ALISYS_DATOSCLIENTE] \
        WHERE DATACREACIO IS NOT NULL AND DATACREACIO > GETDATE()-15 AND CLAVE = \'LLamadaTipoServicio\''

    conn = pyodbc.connect(driver='{ODBC Driver 17 for SQL Server}',server='tcp:khcuy9chlt.database.windows.net,1433',database=bdd, uid=user, pwd=password)

    df = pd.read_sql_query(query, conn)
    
    df = df[['IDCAMPANYA','VALOR','DATACREACIO']]
    
    df['tCreacion'] = pd.to_datetime(df['DATACREACIO'])

    df['Date'] = df['tCreacion'].dt.date
    df['Date'] = df['Date'].astype('str')

    df['dateCreacion'] = df['tCreacion'].dt.date
    df['dateCreacion'] = df['dateCreacion'].astype('str')
    
    df = df[['Date','dateCreacion','IDCAMPANYA','VALOR']]
    
    df = df.groupby(['Date','IDCAMPANYA','VALOR']).count()
    
    df = df.reset_index()
    
    df = df[(df['VALOR'] == 'Estado de Pedido') | (df['VALOR'] == 'No tiene Numero')]
    
    df = df.groupby('Date').sum()[['dateCreacion']]
    
    return df
    

In [3]:
df_calls = getCallData()

In [4]:
df_calls.head()

,dateCreacion
Date,
2021-06-14,91
2021-06-15,113
2021-06-16,113
2021-06-17,102
2021-06-18,132


# Datos Envíos

In [5]:
# Devuelve un df unificado de todos los ficheros cargados
def getDataEnvios(df):
    df = df.iloc[:, [0,4,15]]
    df['Fecha envío'] = pd.to_datetime(df['Fecha envío'])
    dias = list(calendar.day_name)
    df['dayofweek'] = [dias[numero] for numero in df['Fecha envío'].dt.dayofweek]
    df['Fecha envío'] =  pd.to_datetime(df['Fecha envío']).astype('str')
    df = df[['Fecha envío','Código servicio','dayofweek']]
    df.reset_index(inplace=True)
    df = df[['Fecha envío','Código servicio','dayofweek']]
    df['total'] = 1
    encoder = OneHotEncoder(handle_unknown='ignore')
    enc = df[['dayofweek']]
    encoder.fit(enc.values.reshape(-1,1))
    encoded = encoder.transform(enc.values.reshape(-1,1))
    encoded_df = pd.DataFrame(encoded.todense())
    encoded_df.columns = encoder.categories_[0]
    df = pd.concat([df,encoded_df], axis=1)
    df.drop(columns=['Código servicio','dayofweek'], inplace=True)
    df = df.groupby(['Fecha envío']).sum()
    df.Monday = [ 1 if value > 0 else 0 for value in df.Monday]
    df.Tuesday = [ 1 if value > 0 else 0 for value in df.Tuesday]
    df.Wednesday = [ 1 if value > 0 else 0 for value in df.Wednesday]
    df.Thursday = [ 1 if value > 0 else 0 for value in df.Thursday]
    df.Friday = [ 1 if value > 0 else 0 for value in df.Friday]
    df.Saturday = [ 1 if value > 0 else 0 for value in df.Saturday]
    df.Sunday = [ 1 if value > 0 else 0 for value in df.Sunday]
    df = df.drop(labels=['Saturday','Sunday'], axis=1)
    
    return df
    

In [6]:
df4800 = pd.read_excel('data/4800.xlsx')
df4802 = pd.read_excel('data/4802.xlsx')
df4803 = pd.read_excel('data/4803.xlsx')
df4806 = pd.read_excel('data/4806.xlsx')
df4810 = pd.read_excel('data/4810.xlsx')

c:\repos\tfm\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
df_shipping = pd.DataFrame()
df_shipping = pd.concat([df_shipping, df4800])
df_shipping = pd.concat([df_shipping, df4802])
df_shipping = pd.concat([df_shipping, df4803])
df_shipping = pd.concat([df_shipping, df4806])
df_shipping = pd.concat([df_shipping, df4810])

In [8]:
df_shippings = getDataEnvios(df_shipping)

<ipython-input-5-3884b16a4d97>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fecha envío'] = pd.to_datetime(df['Fecha envío'])
<ipython-input-5-3884b16a4d97>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayofweek'] = [dias[numero] for numero in df['Fecha envío'].dt.dayofweek]
<ipython-input-5-3884b16a4d97>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [9]:
df_shippings.head()

,total,Friday,Monday,Thursday,Tuesday,Wednesday
Fecha envío,,,,,,
2021-06-14,3086,0,1,0,0,0
2021-06-15,2461,0,0,0,1,0
2021-06-16,2451,0,0,0,0,1
2021-06-17,2315,0,0,1,0,0
2021-06-18,2043,1,0,0,0,0


# Union de los datos de llamadas 

In [10]:
def dataJoin(df_calls, df_shippins):
    df = pd.concat([df_shippins, df_calls], axis=1).reset_index()
    df['Fecha'] = df['index']
    df.drop('index', axis='columns', inplace=True)
    
    ordenado = ['Fecha', 'total', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'dateCreacion']
    df = df[ordenado]
    
    df.dropna(inplace=True)
    
    return df

In [11]:
df = dataJoin(df_calls, df_shippings)

In [12]:
df.head()

,Fecha,total,Monday,Tuesday,Wednesday,Thursday,Friday,dateCreacion
0,2021-06-14,3086.0,1.0,0.0,0.0,0.0,0.0,91.0
1,2021-06-15,2461.0,0.0,1.0,0.0,0.0,0.0,113.0
2,2021-06-16,2451.0,0.0,0.0,1.0,0.0,0.0,113.0
3,2021-06-17,2315.0,0.0,0.0,0.0,1.0,0.0,102.0
4,2021-06-18,2043.0,0.0,0.0,0.0,0.0,1.0,132.0


# Data Engineering sobre los datos de llamadas y envíos

In [13]:
def dataEngineering(df):
    # Lags de envios
    df['envioslag1'] = df['total'].shift(periods=1)
    df['envioslag6'] = df['total'].shift(periods=6)
    df['envioslag7'] = df['total'].shift(periods=7)
    df['envioslag8'] = df['total'].shift(periods=8)
    
    #Renombrar por claridad
    df['envios'] = df['total']
    df.drop('total', axis='columns', inplace=True)

    # Datos estacionales
    df['sinlag1'] = np.sin(2*np.pi*(1/1)*df.index)
    df['coslag1'] = np.cos(2*np.pi*(1/1)*df.index)

    df['sinlag2'] = np.sin(2*np.pi*(1/2)*df.index)
    df['coslag2'] = np.cos(2*np.pi*(1/2)*df.index)

    df['sinlag3'] = np.sin(2*np.pi*(1/3)*df.index)
    df['coslag3'] = np.cos(2*np.pi*(1/3)*df.index)

    df['sinlag4'] = np.sin(2*np.pi*(1/4)*df.index)
    df['coslag4'] = np.cos(2*np.pi*(1/4)*df.index)

    df['sinlag5'] = np.sin(2*np.pi*(1/5)*df.index)
    df['coslag5'] = np.cos(2*np.pi*(1/5)*df.index)

    df['sinlag6'] = np.sin(2*np.pi*(1/6)*df.index)
    df['coslag6'] = np.cos(2*np.pi*(1/6)*df.index)

    df['sinlag7'] = np.sin(2*np.pi*(1/7)*df.index)
    df['coslag7'] = np.cos(2*np.pi*(1/7)*df.index)
    
    # Lags de llamadas
    df['llamadaslag1'] = df['dateCreacion'].shift(periods=1)
    df['llamadaslag2'] = df['dateCreacion'].shift(periods=2)
    df['llamadaslag3'] = df['dateCreacion'].shift(periods=3)
    df['llamadaslag4'] = df['dateCreacion'].shift(periods=4)
    df['llamadaslag5'] = df['dateCreacion'].shift(periods=5)
    df['llamadaslag6'] = df['dateCreacion'].shift(periods=6)
    df['llamadaslag7'] = df['dateCreacion'].shift(periods=7)
    
    #Renombrar por claridad
    df['llamadas'] = df['dateCreacion']
    df.drop('dateCreacion', axis='columns', inplace=True)
    
    df.dropna(inplace=True)
    
    df  = df.tail(1).iloc[:,1:-1]
    
    df.reset_index(inplace=True)
    df.drop('index', axis='columns', inplace=True)
    
    return df

In [14]:
df = dataEngineering(df)

In [15]:
df.head()

,Monday,Tuesday,Wednesday,Thursday,Friday,envioslag1,envioslag6,envioslag7,envioslag8,envios,...,coslag6,sinlag7,coslag7,llamadaslag1,llamadaslag2,llamadaslag3,llamadaslag4,llamadaslag5,llamadaslag6,llamadaslag7
0,1.0,0.0,0.0,0.0,0.0,2435.0,2043.0,2315.0,2451.0,3832.0,...,-0.5,-4.898587e-16,1.0,92.0,90.0,132.0,113.0,99.0,132.0,102.0


# Normalizacion de datos

In [16]:
def dataScaler(df):
    # Cargar scaler desde Pickle
    scaler_pkl_filename = 'data/scaler.pkl'
    scaler_pkl = open(scaler_pkl_filename, 'rb')
    scaler = pickle.load(scaler_pkl)
    
    df = scaler.transform(df)
    
    return df

In [17]:
df = dataScaler(df)

In [18]:
df

array([[ 1.90743227, -0.50644264, -0.48850421, -0.5207082 , -0.45946829,
         0.03145141, -0.3430039 , -0.07928435,  0.0526206 ,  1.43744789,
         0.63204558,  0.        ,  0.02236815,  0.96395204, -1.22545908,
        -0.71319739,  0.039689  , -1.38895772, -1.36394213,  0.41787743,
         1.27762868, -0.70347195,  0.00966503,  1.2376988 , -1.11438749,
        -1.13480555, -0.72743756, -0.91404212, -1.05616091, -0.73886676,
        -1.03140839]])

# Cargar modelo

In [19]:
def dataPredict(data):
    # Cargar modelo desde Pickle
    regl_pkl_filename = 'data/regl.pkl'
    regl_pkl_model = open(regl_pkl_filename, 'rb')
    regl = pickle.load(regl_pkl_model)
    
    predict = regl.predict(data)
    
    return predict

In [20]:
prediction = dataPredict(df)

In [21]:
prediction

array([98.30151563])

# Streamlit

In [ ]:
%%writefile dashBoard.py

import pandas as pd
import streamlit as st


def my_widget(key): 
    st.subheader('Carga de datos')
    
    #Carga datos Agencia 4800
    data_file = st.file_uploader("Agencia 4800",type=['xlsx'])
    if data_file is not None:
        file_details = {"Filename":data_file.name,"FileType":data_file.type,"FileSize":data_file.size}
        st.write(file_details)
        df = pd.read_excel(data_file)
        st.dataframe(df)
        
    #Carga datos Agencia 4802
    data_file = st.file_uploader("Agencia 4802",type=['xlsx'])
    if data_file is not None:
        file_details = {"Filename":data_file.name,"FileType":data_file.type,"FileSize":data_file.size}
        st.write(file_details)
        df = pd.read_excel(data_file)
        st.dataframe(df)
    
st.header('Predicción llamadas contact center')

st.header('Importar ficheros de datos para predecir')
       
#container = st.beta_container()
#columna1, columna2, columna3 = container.beta_columns(3)


#st.set_page_config(layout="wide")

#st.image('imagen_ruta.jpg',  use_column_width=True)

with st.sidebar:
    clicked = my_widget("Actualizar")
    if clicked:
        pass